In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import os
from torchvision.datasets import ImageFolder

In [4]:
#load path
train_dir = '/home/kevinluo/breast_density_classification/breast_density_data/train'
batch_size = 32
num_classes = 4

train_dataset = datasets.ImageFolder(
    root=train_dir,
    transform=transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)
 
# define the model architectures from pre-trained model (I use resnet18 to train the regression model)
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

NameError: name 'datasets' is not defined

In [2]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import os

# loading data
train_dir = '/home/kevinluo/breast_density_classification/datasets/train'
batch_size = 16
num_classes = 4
train_dataset = ImageFolder(
    root=train_dir,
    transform=transforms.Compose([
        transforms.Resize(640),
        transforms.CenterCrop(640),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8
)
test_dir = "/home/kevinluo/breast_density_classification/datasets/test"
test_dataset = ImageFolder(
    root=test_dir,
    transform=transforms.Compose([
        transforms.Resize(640),
        transforms.CenterCrop(640),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4
)
# define our model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

/home/kevinluo/anaconda3/envs/yolov8/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/kevinluo/anaconda3/envs/yolov8/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
# modify model to Regression model
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, num_classes),
    nn.Softmax(dim=1)
)

# define the loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# define training and testing function
def train(model, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 10 == 9:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))
            running_loss = 0.0


def test(model, test_loader):
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy: %d %%' % (100 * correct / total))

# train and test model
num_epochs = 50
for epoch in range(num_epochs):
    train(model, train_loader, criterion, optimizer)
    test(model, test_loader)

In [ ]:
import matplotlib.pyplot as plt

# 定义空列表来存储训练损失和准确率
train_losses = []
train_accs = []

# 迭代训练
for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0

    # 训练一个epoch
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 计算训练损失
        train_loss += loss.item() * inputs.size(0)

        # 计算训练准确率
        _, preds = torch.max(outputs, 1)
        train_acc += torch.sum(preds == labels.data)

    # 计算平均训练损失和准确率
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc / len(train_loader.dataset)

    # 将训练损失和准确率添加到列表中
    train_losses.append(train_loss)
    train_accs.append(train_acc)

    # 在每个epoch结束时输出训练损失和准确率
    print('Epoch: {} \tTraining Loss: {:.6f} \tTraining Accuracy: {:.6f}'.format(epoch+1, train_loss, train_acc))

# 画出训练损失和准确率的图像
plt.plot(train_losses, label='Training Loss')
plt.plot(train_accs, label='Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Metric')
plt.legend()
plt.show()
